![](http://community.dur.ac.uk/jeroen.van-hunen/ASPECT/logos.png) 

This page will introduce the geodynamical software ASPECT and its postprocessing. The most recent copy of the very extensive and useful ASPECT user guide is available  [here](http://www.math.clemson.edu/~heister/manual.pdf). 

In the following I will assume that you will be able to log into Durham's Hamilton HPC cluster. If not, please refer to the page [Accessing Hamilton from your own computer](./Hamilton_from_own_computer.html). Furthermore, ASPECT needs to be installed already. For more information on how to install ASPECT, please see [ASPECT installation](./ASPECT_installation.html).

## Running calculations on Hamilton
The first time we are going to run a calculation, we need to create a folder from where to run the model calculations. To store the data in a location with enough capacity, we will use again the `/ddn/data/^username^/` folder (where `^username^` should be replaced with your CIS username. So log into hamilton again, and run the following commands:

    cd /ddn/data/^username^
    mkdir aspect-data
    cd aspect-data

You'll only have to do this once. So next time, you log onto hamilton, you can go straight to the data folder:

    cd /ddn/data/^username^/aspect-data

Now you will need to copy over the aspect executable from the previous section to your local working folder:

In [3]:
cd /ddn/data/^username^/aspect-data
cp ^location-of-aspect-executable^/^aspect-executable-name^ .

SyntaxError: invalid syntax (<ipython-input-3-c43729515e1c>, line 1)

You'll also need to create an ASPECT input file, or copy one over to this folder. For the first ASPECT calculation, I suggest we use the following input file, called Blbach1a.prm.

In [4]:
cp /ddn/data/dgl6jv1/public/Blbach1a.prm .
nano Blbach1a.prm

SyntaxError: invalid syntax (<ipython-input-4-e51d5592bd53>, line 1)

This will open an *editor* in Hamilton, which, in this case, can be used to add or change text to the file `Blbach1a.prm`. Now highlight the text on the browser page, copy it (control-C), and paste it into the editor. Now save the file (control-O and then control-X).

Now we have an input file for a simple convection calculation in a square unit-size box for a given Rayleigh number. The vigour of convection and resulting heat flow at the surface of the box have been calculated by many different modellers, and published in a classic paper by Blankenbach et al. (1989), and can be found [here](https://academic.oup.com/gji/article/98/1/23/622167). So we know rather exactly what the results of this calculation should be, and our ASPECT calculation should be able to give similar results. 

Next, we would like to run the calculation on Hamilton. If you've done the short course on Hamilton usage, you'll now know that you'll need a submission script. A example can be copied over and opened as follows: 

In [5]:
cp /ddn/data/dgl6jv1/public/mpi.slurm .
nano mpi.slurm

SyntaxError: invalid syntax (<ipython-input-5-b09d80d3fc26>, line 1)

Now we need to make a few changes to the file. So scroll down to the end of the file. The line 

In [6]:
#SBATCH -n 8

refers to the number of processors used in the calculation. Change the number if you like. But keep in mind that the number should correspond to the number in the last line:

In [7]:
mpirun -np 8 ./aspect.rel $1

SyntaxError: invalid syntax (<ipython-input-7-2844803b13c9>, line 1)

In this last line, the text `./aspect.rel` refers to the name of the ASPECT executable, which is, in this case, named `aspect.rel` and placed in the folder `.` (which is the folder you are presently located in). If you named your executable differently, you'll need to change it here.

Finally, we need to specify the job queue in which we want to place the calculation. There are three useful queues we can use, and they are named `par6.q`, `par7.q`, and `test.q`. Which one to choose depends on the type of model: `par6.q` and `par7.q` are normally used for larger calculations lasting more than an hour (up to 72 hours). But if you have a small calculation running less than an hour, you can choose the `test.q` queue, which allows a maximum of 12 processors, and will automatically stop if the calculation lasts longer than an hour. The queue choice may also depend on how musy each of the queues is. Use 

In [8]:
sfree

NameError: name 'sfree' is not defined

to see how many processors are available in each queue. Once you are happy with the modifications of the `mpi.slurm` file, close it. 

Finally, we need to copy over the aspect executable to the folder where we are running the calculations. Check where your aspect executable `aspect.rel` was created (if you followed Phil's installation guide, it is probably in `/ddn/data/^username^/install/aspect/`):

In [9]:
cp /ddn/data/^username^/install/aspect/aspect.rel .

cp: /ddn/data/^username^/install/aspect/aspect.rel: No such file or directory


Now it is time to run the calculation, by typing the following commands:  

In [10]:
sbatch mpi.slurm Blbach1a.prm

SyntaxError: invalid syntax (<ipython-input-10-d47edf83ab72>, line 1)

If all goes well, you will be returned a job ID number. Whether the job will start running immediately or not depends on the availability of free processors. You can find out the *status* of all your jobs in the queue by typing

In [11]:
squeue -u ^username^

SyntaxError: invalid syntax (<ipython-input-11-06fdea6debdf>, line 1)

where a status `R` means the job is running, and `QW` means it is still queueing and waiting for the next available set of processors slots. 

Once the code starts running, it will create a folder to store the results in. The name of the folder is given in the input file. In this example, the folder name is `Blbach1a`. As you see, I named it the same as the input file, but without the `.prm` suffix, so that it is easy to match the input file name and the matching folder name. But you are essentially free to choose whatever folder name you like. 

The output of the code is stored in two places: a) in a file called `slurm-`^jobID^`.out`, where ^jobID^ refers to the job ID number of the calculation, and b) in the file `Blbach1a/log.txt`. So to check the output that the code is procuding while it is running, you could type:

In [12]:
tail -f Blbach1a/log.txt

SyntaxError: invalid syntax (<ipython-input-12-4fd191b2facd>, line 1)

The results folder contains a number of other files and folders with information. For example, a subfolder with further output for visualisation using the visualisation software *Paraview* is in the subfolder `solution` (see further details on that below). 

Perhaps one of the more useful files is the `statistics` file, that can list several physical quantities (like maximum velocity or average temperature) for every time step of the calculation, but also a lot of more technical information, like the number of iterations needed to find the solution for that timestep. The first part of an example of the `statistics` is shown in the figure below. The first 22 lines are preceded by a `#`, and are 'header lines', that contain info on the data to follow below. It describes what data is available in each of the 22 columns below the header lines.
![](http://community.dur.ac.uk/jeroen.van-hunen/ASPECT/screenshot_statistics_file.png)

## Visualising results
### Paraview 
The most useful tool to visualize results from the calculations is `paraview`. It is already installed on Hamilton, and you can use it by first loading it, and then giving the command to start it:

In [13]:
module load paraview/4.3.1
paraview&

SyntaxError: invalid syntax (<ipython-input-13-14df370b2fd9>, line 1)

This should now start the graphical user interface. Paraview versions all look slightly different with different versions and on different platforms, but the basic functionality should look very similar. There are a huge number of options with this software, but here we'll just show you the basics to get a first plot produced. Extensive tutorials can be downloaded [here](https://www.paraview.org/Wiki/The_ParaView_Tutorial). To create a first plot, do the following:
- click on the 'Open' folder button in the top-left
- browse through your folders and go to the subfolder `solution` in the folder where the data for the calculation is stored (in this example `Blbach1a`)
- choose the file `solution-..pvtu`
- click the blue 'Apply' button: now a grey box should appear
- in the drop-down box that says 'Solid Color', choose a quantity you want to visualize: I suggest to take 'T' first, to visualize the temperature field. 
- Now press the green 'Go' button to start time-stepping through the available snapshots in time.
You should see something similar to the plot below: 
![](http://community.dur.ac.uk/jeroen.van-hunen/ASPECT/paraview_screenshot.png)
There is not an awful lot to see in this case, as there are only a few time snapshots stored to file, but in future calculations, looking at the results as an 'animation' will be useful. The options for improving the plots are almost endless, and a few things to add are contours and velocity vectors.

### Gnuplot
Simple line plots (but also much more complex and fancy plots) can be easily made with gnuplot. A basic manual and tutorial can be found [here](https://people.duke.edu/~hpgavin/gnuplot.html), while the main gnuplot webpage (with tutorials, download info and much more) can be found [here](http://www.gnuplot.info/). 

To get started with a simple example, type the following from the `/ddn/data/^username^/aspect-data` folder:

In [14]:
module load gnuplot/current
gnuplot

SyntaxError: invalid syntax (<ipython-input-14-a9d1f89a4f08>, line 1)

This will produce a `gnuplot >` prompt, and you can now give gnuplot commands. For example, we can plot two of the coumns of the `statistics` file discussed above: 

In [15]:
plot "Blbach1a/statistics" using 2:12 title 'vrms'

SyntaxError: invalid syntax (<ipython-input-15-755340be6d13>, line 1)

You should see something like this: 
![](http://community.dur.ac.uk/jeroen.van-hunen/ASPECT/gnuplot.png)

## Logging on and using Hamilton from your own computer
Using your own laptop or desktop to connect interact with Hamilton is described [here](http://community.dur.ac.uk/jeroen.van-hunen/ASPECT/Hamilton_from_own_computer.html).